In [1]:
# Imports
import pandas as pd
from datetime import datetime, timedelta
import time, requests
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
# Functions
def epoch_to_datetime(epoch_date: int):
    if len(str(epoch_date)) > 10:
        epoch_date = str(epoch_date)[:-3]
    timeArray = time.localtime(int(epoch_date))
    return time.strftime("%Y-%m-%d %H:%M:%S", timeArray)

def get_quotes(symbol: str, interval: str, time_period: int):
    url = "{}/api/v3/klines?symbol={}&interval={}".format("https://api.binance.com", symbol, interval)
    df_quote = pd.DataFrame(requests.get(url).json())
    df_quote.columns = ['open time', 'open', 'high', 'low', 'close', 'volume', 'close time', 'asset volume', 'number of trades', 'taker buy asset volume', 'taker buy quote volume', 'ignore']

    df_quote['open time'] = df_quote['open time'].astype(str)
    df_quote['close time'] = df_quote['close time'].astype(str)

    for index, row in df_quote.iterrows():
        df_quote.at[index,'open time']= str(epoch_to_datetime(int(row['open time'])))
        df_quote.at[index,'close time']= str(epoch_to_datetime(int(row['close time'])))
    return df_quote

def get_open_interest(symbol: str, interval: str):
    url = "{}/futures/data/openInterestHist?symbol={}&period={}&limit=500".format("https://fapi.binance.com", symbol, interval)
    df_quote = pd.DataFrame(requests.get(url).json())
    df_quote.columns = ['symbol', 'OI', 'OI in $', 'date']
    df_quote['date'] = df_quote['date'].astype(str)

    for index, row in df_quote.iterrows():
        df_quote.at[index,'date']= str(epoch_to_datetime(int(row['date'])))
    return df_quote

def get_top_accounts_long_short_accounts(symbol: str, interval: str):
    url = "{}/futures/data/topLongShortAccountRatio?symbol={}&period={}".format("https://fapi.binance.com", symbol, interval)
    df_quote = pd.DataFrame(requests.get(url).json())
    df_quote.columns = ['symbol', 'long', 'short', 'long/short', 'date']
    df_quote['date'] = df_quote['date'].astype(str)

    for index, row in df_quote.iterrows():
        df_quote.at[index,'date']= str(epoch_to_datetime(int(row['date'])))
    return df_quote


def get_top_accounts_long_short_positions(symbol: str, interval: str):
    url = "{}/futures/data/topLongShortPositionRatio?symbol={}&period={}".format("https://fapi.binance.com", symbol, interval)
    df_quote = pd.DataFrame(requests.get(url).json())
    df_quote.columns = ['symbol', 'long', 'long/short ratio', 'short', 'date']
    df_quote['date'] = df_quote['date'].astype(str)

    for index, row in df_quote.iterrows():
        df_quote.at[index,'date']= str(epoch_to_datetime(int(row['date'])))
    return df_quote

def get_global_long_short_account_ratio(symbol: str, interval: str):
    url = "{}/futures/data/globalLongShortAccountRatio?symbol={}&period={}".format("https://fapi.binance.com", symbol, interval)
    df_quote = pd.DataFrame(requests.get(url).json())
    df_quote.columns = ['symbol', 'long', 'long/short','short', 'date']
    df_quote['date'] = df_quote['date'].astype(str)

    for index, row in df_quote.iterrows():
        df_quote.at[index,'date']= str(epoch_to_datetime(int(row['date'])))
    return df_quote


def get_taker_long_short_ratio(symbol: str, interval: str):
    url = "{}/futures/data/takerlongshortRatio?symbol={}&period={}".format("https://fapi.binance.com", symbol, interval)
    df_quote = pd.DataFrame(requests.get(url).json())
    df_quote.columns = ['buy/sell ratio', 'sell volume', 'buy volume', 'date']
    df_quote['date'] = df_quote['date'].astype(str)

    for index, row in df_quote.iterrows():
        df_quote.at[index,'date']= str(epoch_to_datetime(int(row['date'])))
    return df_quote

In [3]:
# Select ticker
symbol = "BTCUSDT"
number_of_days_to_track = 365
interval = "1d"  # 1d 4h 1h 


In [4]:
df_data = get_quotes(symbol, interval, number_of_days_to_track)
df_data_oi = get_open_interest(symbol, interval)
df_top_accounts_long_short = get_top_accounts_long_short_positions(symbol, interval)
df_global_accounts_long_short = get_global_long_short_account_ratio(symbol, interval)
df_taker_long_short_ratio = get_taker_long_short_ratio(symbol, interval)

In [14]:
# Volume
fig = px.line(df_data, x="open time", y="volume", title="{} Binance Volume".format(symbol), template = "plotly_dark")
fig.show()

In [15]:

# Number of trades
fig = px.line(df_data, x="open time", y="number of trades", title="{} Binance Number of trades".format(symbol), template = "plotly_dark")
fig.show()

In [7]:
# Price spreads
fig = px.line(df_data, x="open time", y=["low","high","close"], title='{} Binance Spread High Low Close'.format(symbol), template = "plotly_dark")
fig.update_layout(legend_title="Price")
fig.show()

In [8]:
# Open Interest
fig = px.line(df_data_oi , x="date", y="OI", title="{} Binance Open Interest".format(symbol), template = "plotly_dark")
fig.show()

In [9]:
# Open Interest
fig = px.line(df_data_oi , x="date", y="OI in $", title="{} Binance Open Interest in $ value".format(symbol), template = "plotly_dark")
fig.show()

In [10]:
fig = px.line(df_top_accounts_long_short , x="date", y="long/short ratio", title="{} Binance Top Accounts Long/Short Ratio".format(symbol), template = "plotly_dark")
fig.show()

In [11]:
# Global Accounts Long/Short ratio
fig = px.line(df_global_accounts_long_short , x="date", y="long/short", title="{} Binance Global Accounts Long/Short Ratio".format(symbol), template = "plotly_dark")
fig.show()

In [12]:
# taker Buy / Sell Ratio
fig = px.line(df_taker_long_short_ratio, x="date", y="buy/sell ratio",title="{} Binance Taker Buy / Sell ratio".format(symbol), template = "plotly_dark")
fig.show()

In [13]:
# Taker Buy / Sell Volume
fig = px.area(df_taker_long_short_ratio, x="date", y=["buy volume","sell volume"], title='{} Binance Taker Buy / Sell Volume'.format(symbol), template = "plotly_dark")
fig.update_layout(legend_title="Price")
fig.show()